# <b>0. IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2 as cv

# <b> 1. Paths to directories and files

### 1.1 Donwloading the MNIST dataset from TF and preparing the directories to save the new dataset

In [6]:
#donwload dataset mnist
(x_train_ori, y_train_ori), (x_test_ori, y_test_ori) = tf.keras.datasets.mnist.load_data()
assert x_train_ori.shape == (60000, 28, 28)
assert x_test_ori.shape == (10000, 28, 28)
assert y_train_ori.shape == (60000,)
assert y_test_ori.shape == (10000,)

In [7]:
#files path in a dictionary
paths = dict()

paths['main'] = os.path.dirname('D:\Javi\Python\MNIST_OD')
paths['dataset'] = os.path.join(paths['main'], 'MNIST_dataset')
paths['train_data'] = os.path.join(paths['dataset'], 'train')
paths['test_data'] = os.path.join(paths['dataset'], 'test')
# paths['test_img'] = os.path.join(paths['test_data'], 'test')

### 1.2 Preparing the dataset to generate new data

In [23]:
#convert the raw data to a dataset-object

train = tf.data.Dataset.from_tensor_slices((x_train_ori,y_train_ori))
test = tf.data.Dataset.from_tensor_slices((x_test_ori,y_test_ori))

#genetating the input data format
BATCH = 32
FETCH = 2
SHUFFLE = 600


# train = train.batch(BATCH).shuffle(SHUFFLE)#.prefetch(FETCH)
# test = test.batch(BATCH)#.prefetch(FETCH)

# 2. Functions to preproccess images in to the new dataset

In [3]:
#functions to rotate images

def rotate_image(image, angle, not_print = True):
    '''
    This function rotate the image and plot it
    Imput params:
        image = array like,
        angle = counterclockwise rotate angle (degrees),
        not_print = if 'True' do not plot an image
    Return:
        image = image rotated [array like]
    '''
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
    if not not_print:
        plt.imshow(result,cmap='gray')
        plt.show()
    return result



In [30]:
train.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(28, 28), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>